In [23]:
import sys
sys.path.append("../")
import API_keys
from Global_variable import *

In [86]:
from langchain_community.document_loaders.pdf import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.agents import Tool
import fitz

def extract_first_page(pdf_path: str) -> str:
    print('/////////////////////////////')
    pdf_document = fitz.open(pdf_path)
    first_page = pdf_document.load_page(0)
    # text = first_page.get_text()
    # if text.strip():
    #     print('////////////////////', text)
    #     return text
    temp_path = AbsoluteBotPath + '/Knowledge/Books/PDF/temp.pdf'
    new_pdf_document = fitz.open()
    new_pdf_document.insert_pdf(pdf_document, from_page=0, to_page=0)
    new_pdf_document.save(AbsoluteBotPath + '/Knowledge/Books/PDF/temp.pdf')
    new_pdf_document.close()
    pdf_document.close()
    pdf_loader = PyMuPDFLoader(temp_path, extract_images=True)
    child_doc = pdf_loader.load()
    print('////////', child_doc[0].page_content)
    return child_doc[0].page_content
scan_first_page_tool=[
    Tool(
        name="scan_first_page",
        func=extract_first_page,
        description="quét thông tin bìa cuốn sách",
    )
]

In [87]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools,return_intermediate_steps=True)
    return executor

In [104]:
SCAN_PROMPT = """Bạn là một trợ lí thông minh chuyên phân tách thông tin cần thiết từ văn bản. Hãy phân tách đoạn văn bản được đưa vào 
                    theo các mục: Tác giả, Nhà xuất bản, Năm xuất bản, Call no. Nếu không tìm thấy kết quả cho mục nào thì hãy để mục đó là None.
                    Nếu dùng đoạn văn bản đầu vào vẫn không tìm thấy đầy đủ thông tin cần thiết thì bạn có thể dùng tool scan_first_page (đầu vào của tool này là pdf_path cũng được cung cấp trong văn bản đầu vào) để quét thông tin bìa sách
                    và dùng nó để tiếp tục phận tích thông tin. Lưu ý hãy cố gắng tìm thông tin trong văn bản đầu vào trước, nếu thực sự không tìm được thì mới dùng tool vì dùng tool này rất tốn tài nguyên, và đừng thêm các kí tự đặc biệt '\n' vào trong output.
                    Hãy làm theo các ví dụ sau đây:
                    *Ví dụ với văn bản đầu vào:
                        -Input: 'Giáo trình giải tích: Trần Đức Long, Hoàng Quốc Toàn, Nguyễn Đình Sang T2 Phép tính vi phân hàm một biến dãy hàm chuỗi hàm/. -- Hà Nội: Đại Học Quốc Gia Hà Nội, 2001. - 200tr.; 21cm.
                                Sách có tại Phòng Mượn, Thư viện Đại học Sư phạm Kỹ thuật TP. Hồ Chí Minh.
                                Số phân loại: 515.07 T772-L848'
                        -Output: 'Tác giả': 'Trần Đức Long, Hoàng Quốc Toàn, Nguyễn Đình Sang', 'Nhà xuất bản': 'Đại Học Quốc Gia Hà Nội', 'Năm xuất bản': '2001', 'Call no': '515.07 T772-L848'
                    *Ví dụ với thông tin bìa sách:
                        -Input: 
                            + Mô tả: Thời kỳ cổ đại và phong kiến Call no: 515.07 T772-L848
                            " Thông tin bìa sách: TRINH QUANGVU\nLUOC SU\nSKV014673*\nNHA XUAT BAN VAN HOA - THONG TIN
                        -Output: 'Tác giả': 'Trịnh Quang Vũ', 'Nhà Xuất bản': 'Vắn hóa - Thông tin', 'Call no': ' 515.07 T772-L848'
                    Sau khi đã trích xuất được các thông tin thì trả về một json với đầỳ đủ các key: 'Tác giả', 'Nhà xuất bản', 'Năm xuất bản', 'Call no' và các giá trị tương ứng được tìm thấy ở bên trên"""
                    


In [105]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
extract_info_agent = create_agent(llm, scan_first_page_tool, SCAN_PROMPT)

In [106]:
mess = """Giáo trình đo đạc địa chính: Dùng trong các trường THCN/ Nguyễn Trọng San. -- H.: Nxb Hà Nội, 2006
412tr; 24cm


Dewey Class no. : 526 -- dc 22
Call no. : 526 N573-S194
pdf_path: /home/xuanai/LLM/Bot/Knowledge/Books/PDF/gt0125641_2238.pdf"""
extract_info_agent.invoke({'messages': [mess]})

/////////////////////////////


FileNotFoundError: no such file: ' /home/xuanai/LLM/Bot/Knowledge/Books/PDF/gt0125641_2238.pdf'

In [107]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
system_prompt = ("""Bạn là một trợ lí thông minh chuyên phân tách thông tin cần thiết từ văn bản. Hãy phân tách đoạn văn bản được đưa vào 
                    theo các mục: Tác giả, Nhà xuất bản, Năm xuất bản, Call no. Nếu không tìm thấy kết quả cho mục nào thì hãy để mục đó là None.
                    Hãy làm theo các ví dụ sau đây:
                    *Ví dụ:
                        -Input: 'Giáo trình giải tích: Trần Đức Long, Hoàng Quốc Toàn, Nguyễn Đình Sang T2 Phép tính vi phân hàm một biến dãy hàm chuỗi hàm/. -- Hà Nội: Đại Học Quốc Gia Hà Nội, 2001. - 200tr.; 21cm.
                                Sách có tại Phòng Mượn, Thư viện Đại học Sư phạm Kỹ thuật TP. Hồ Chí Minh.
                                Số phân loại: 515.07 T772-L848'
                        -Output: 'Tác giả': 'Trần Đức Long, Hoàng Quốc Toàn, Nguyễn Đình Sang', 'Nhà xuất bản': 'Đại Học Quốc Gia Hà Nội', 'Năm xuất bản': '2001', 'Call no': '515.07 T772-L848'""")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = (
    prompt
    | llm
)

In [113]:
anwser = chain.invoke({'messages': [mess]})
out = anwser.content

In [118]:
import ast
dict = ast.literal_eval(out)
dict

{'Tác giả': 'Nguyễn Trọng San',
 'Nhà xuất bản': 'Nxb Hà Nội',
 'Năm xuất bản': '2006',
 'Call no': '526 N573-S194'}

In [ ]:
def process(description):
    anwser = chain.invoke({'messages': [description]})
    out = anwser.content
    dict = ast.literal_eval(out)
    return dict

In [17]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io

def extract_text_from_first_page(pdf_path):
    pdf_document = fitz.open(pdf_path)
    first_page = pdf_document.load_page(0)
    text = first_page.get_text()

    if text.strip():
        # If text is found, return it
        return text
    else:
        # If no text is found, check if the page is an image and perform OCR
        image_list = first_page.get_images(full=True)
        
        if image_list:
            xref = image_list[0][0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            
            # Perform OCR on the image
            text = pytesseract.image_to_string(image)
            return text
        else:
            return "No text or image found on the first page."

pdf_path = '/home/xuanai/LLM/Bot/Knowledge/Books/PDF/skv014673_4132.pdf'
text = extract_text_from_first_page(pdf_path)
print(f"Extracted text from the first page:\n{text}")


Extracted text from the first page:
TRINH QUANG VU

  

    
 

2

MLA |g

  



In [19]:
from langchain_community.document_loaders import PyMuPDFLoader
import fitz  # PyMuPDF

def extract_text_from_first_page(pdf_path):
    # Initialize the PyMuPDFLoader with the PDF path
    loader = PyMuPDFLoader(file_path=pdf_path)
    
    # Load the document
    document = loader.load()
    
    # Get the first page
    first_page = document[0]
    
    # Extract text from the first page
    text = first_page.page_content
    
    # Check if text is empty
    if not text.strip():
        # If text is empty, check if the page contains images
        pdf_document = fitz.open(pdf_path)
        page = pdf_document.load_page(0)
        image_list = page.get_images(full=True)
        
        if image_list:
            # If images are present, extract the image
            xref = image_list[0][0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            
            # For now, we don't use OCR; just note that an image was found
            text = "Image found on the first page, but no OCR applied."
        else:
            text = "No text or image found on the first page."

    return text

pdf_path = '/home/xuanai/LLM/Bot/Knowledge/Books/PDF/skv014673_4132.pdf'
text = extract_text_from_first_page(pdf_path)
print(f"Extracted text from the first page:\n{text}")

Extracted text from the first page:
Image found on the first page, but no OCR applied.


In [39]:

from langchain_community.document_loaders.pdf import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
def extract_first_page(pdf_path):
    pdf_document = fitz.open(pdf_path)
    first_page = pdf_document.load_page(0)
    text = first_page.get_text()
    if text.strip():
        return text
    temp_path = AbsoluteBotPath + '/Knowledge/Books/PDF/temp.pdf'
    new_pdf_document = fitz.open()
    new_pdf_document.insert_pdf(pdf_document, from_page=0, to_page=0)
    new_pdf_document.save(AbsoluteBotPath + '/Knowledge/Books/PDF/temp.pdf')
    new_pdf_document.close()
    pdf_document.close()
    pdf_loader = PyMuPDFLoader(temp_path, extract_images=True)
    child_doc = pdf_loader.load()
    return child_doc[0].page_content

In [41]:
pdf_path = '/home/xuanai/LLM/Bot/Knowledge/Books/PDF/UTE. Báo cáo, Đồ án/skl012092_9017.pdf'
pdf_path = '/home/xuanai/LLM/Bot/Knowledge/Books/PDF/skv014673_4132.pdf'
extract_first_page(pdf_path)

'TRINH QUANGVU\nLUOC SU\nSKV014673*\nNHA XUAT BAN VAN HOA - THONG TIN'

In [125]:
import json

def extract_authors_from_json(file_path, key='Tên sách'):
    # Initialize an empty list to store the authors
    authors = set()
    # Open and read the JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)['book_infos']
        for d in data:
            authors.add(d[key])
    return authors

# Specify the path to your JSON file
file_path = '/home/xuanai/LLM/Bot/Knowledge/Books/Json/sample2.json'

# Extract all values with the key 'Tác giả'
authors_list = extract_authors_from_json(file_path)

# Print the list of authors
print(len(authors_list))


16


In [2]:
subjects_with_numbers = ['Sinh học (740)', 'Hóa học (688)', 'Toán học ']

# Function to clean each subject string
def clean_subject(subject):
    # Find the index of the first '(' and take the substring before it
    cleaned_subject = subject.split(' (')[0]
    # Strip any extra whitespace from the ends
    cleaned_subject = cleaned_subject.strip()
    return cleaned_subject

# Apply the cleaning function to each element in the list
cleaned_subjects = [clean_subject(subject) for subject in subjects_with_numbers]
cleaned_subjects

['Sinh học', 'Hóa học', 'Toán học']

In [4]:
clean_subject('UTE. Thông tin -Thư viện')

'UTE. Thông tin -Thư viện'